In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("marshuu/dog-breeds")

print("Path to dataset files:", path)
dogs_by_state = pd.read_csv(path + '\dog_breeds.csv')

In [ ]:
display(dogs_by_state.head())

In [ ]:
fig = px.choropleth(data_frame=dogs_by_state,
                    locations='Country of Origin',
                    locationmode='country names', # tells Plotly to use full country names
                    color='Breed',             # value to be mapped with color
                    color_continuous_scale='Viridis',  # color theme
                    title='Choropleth Map by Country')

fig.show()


In [ ]:
data_psy_complet = pd.read_excel('data-for-publication.xlsx', sheet_name='Dataset')
data_psy_information = pd.read_excel('data-for-publication.xlsx', sheet_name='Information')

In [ ]:

print(data_psy_complet.columns)

In [ ]:

sterilized = data_psy_complet.query("sterilization_status == 'sterilized'")
not_sterilized = data_psy_complet.query("sterilization_status == 'intact'")
print('Impulsion of sterilized dogs')
print(sterilized['impulsive'].mean())
print(sterilized['Impulsivity_age'].mean())
print('Impulsion of intact dogs')
print(not_sterilized['impulsive'].mean())
print(not_sterilized['Impulsivity_age'].mean())
print()
print('Dominance of sterilized dogs')
print(sterilized['Dominant'].mean())
print('Dominance of intact dogs')
print(not_sterilized['Dominant'].mean())
print()
print('Patience of sterilized dogs')
print(sterilized['Patient'].mean())
print('Patience of intact dogs')
print(not_sterilized['Patient'].mean())
print()
print('Patience of sterilized dogs')
print(sterilized['Patient'].mean())
print('Patience of intact dogs')
print(not_sterilized['Patient'].mean())
print(not_sterilized['Patient'].mean())


In [ ]:
max = 0
atribut = ''
nove_data_psy = data_psy_complet.drop('dog.id', axis=1)
nove_data_psy.drop('Separation_age', axis=1, inplace=True)
numeric_cols = nove_data_psy.select_dtypes(include='number').columns
for i in numeric_cols:

    x = not_sterilized[str(i)].mean()
    y = sterilized[str(i)].mean()
    if abs(x - y) > max:
        max = abs(x - y)
        atribut = i


In [ ]:
print(atribut)
print(max)

In [ ]:
data_psy_breeder = data_psy_complet.query("obtained_from == 'breeder'")
data_psy_not_breeder = data_psy_complet.query("obtained_from != 'breeder'")
print(data_psy_complet['Impulsivity_age'].mean())
old_dogs = data_psy_complet.query("Impulsivity_age > 5.21")
print(old_dogs["impulsive"].mean())
young_dogs = data_psy_complet.query("Impulsivity_age < 5.21")
print(young_dogs["impulsive"].mean())

print('====================')
print('form breeder')
print('====================')

print(data_psy_breeder['Impulsivity_age'].mean())
old_dogs_breeder = data_psy_breeder.query("Impulsivity_age > 5.21")
print(old_dogs_breeder["impulsive"].mean())
young_dogs_breeder = data_psy_breeder.query("Impulsivity_age < 5.21")
print(young_dogs_breeder["impulsive"].mean())

print('====================')
print('not form breeder')
print('====================')

print(data_psy_not_breeder['Impulsivity_age'].mean())
old_dogs_not_breeder = data_psy_not_breeder.query("Impulsivity_age > 5.21")
print(old_dogs_not_breeder["impulsive"].mean())
young_dogs_not_breeder = data_psy_not_breeder.query("Impulsivity_age < 5.21")
print(young_dogs_not_breeder["impulsive"].mean())


Working with information dataset to extract values rated on the scale of 1 to 5

In [ ]:
data_psy_information['SECTION'] = data_psy_information['SECTION'].fillna(method='ffill')
target_text = """1 = Never
2 = Rarely
3 = Sometimes
4 = Often
5 = Always or almost always"""
info_rated = data_psy_information[data_psy_information['COLUMN VALUES'] == target_text]
print(info_rated.head())

In [ ]:
max = 0
max_sterilized = 0
max_intact = 0
max_column = ''
ranked_data = pd.DataFrame([{
    'Attribute': [],
    'Absolute difference': [],
    'Sterilized mean': [],
    'Intact mean': [],
    'Bigger value is': []
}])

for i in info_rated['COLUMN NAME']:
    ster = sterilized[i].mean()
    not_ster = not_sterilized[i].mean()
    if ster > not_ster:
        bigger_value = 'sterilized'
    else:
        bigger_value = 'intact'
    ranked_data.loc[len(ranked_data)] = [i, abs(ster - not_ster), ster, not_ster, bigger_value]
    if abs(ster - not_ster) > max:
        max = abs(ster - not_ster)
        max_sterilized = ster
        max_intact = not_ster
        max_column = i

print('Column:', max_column)
print('absolute difference:', max)
print('sterilized rating:', max_sterilized)
print('intact rating:', max_intact)
ranked_data = ranked_data.drop(index=0)


Sort values by biggest to smallest

In [ ]:
ranked_data = ranked_data.sort_values(by='Absolute difference', ascending=False)
print(ranked_data.head())

Make different tables for sterilized and intact

In [ ]:
ranked_sterilized = ranked_data[ranked_data['Bigger value is'] == 'sterilized']
ranked_intact = ranked_data[ranked_data['Bigger value is'] == 'intact']

In [ ]:
intact_barplot = sns.barplot(data = ranked_intact[:5], x='Attribute', y='Absolute difference')
sterilized_barplot = sns.barplot(data = ranked_sterilized[:5], x='Attribute', y='Absolute difference')
plt.xticks(rotation=45, fontsize=7)
plt.title("Most differing attributes by sterilization status")
plt.xlabel("")

In [ ]:
all_barplot = sns.barplot(data = ranked_data, x='Attribute', y='Absolute difference', hue='Bigger value is')
all_barplot.tick_params(axis='x', which='both', bottom=False, labelbottom=False)
plt.title("Biggest trait differences - All")


Grafy pre sterilizáciu konkrétne u samíc

In [ ]:
data_samice = data_psy_complet[data_psy_complet['sex'] == 'female']
data_samce = data_psy_complet[data_psy_complet['sex'] == 'male']

data_ster_samice = data_samice[data_samice['sterilization_status'] == 'sterilized']
data_notster_samice = data_samice[data_samice['sterilization_status'] == 'intact']
data_ster_samce = data_samce[data_samce['sterilization_status'] == 'sterilized']
data_notster_samce = data_samce[data_samce['sterilization_status'] == 'intact']


In [ ]:
max = 0
max_sterilized = 0
max_intact = 0
max_column = ''
ranked_data_samice = pd.DataFrame([{
    'Attribute': [],
    'Absolute difference': [],
    'Sterilized mean': [],
    'Intact mean': [],
    'Bigger value is': []
}])

for i in info_rated['COLUMN NAME']:
    ster = data_ster_samice[i].mean()
    not_ster = data_notster_samice[i].mean()
    if ster > not_ster:
        bigger_value = 'sterilized'
    else:
        bigger_value = 'intact'
    ranked_data_samice.loc[len(ranked_data_samice)] = [i, abs(ster - not_ster), ster, not_ster, bigger_value]
    if abs(ster - not_ster) > max:
        max = abs(ster - not_ster)
        max_sterilized = ster
        max_intact = not_ster
        max_column = i
ranked_data_samice = ranked_data_samice.drop(index=0).sort_values(by='Absolute difference', ascending=False)

In [ ]:
max = 0
max_sterilized = 0
max_intact = 0
max_column = ''
ranked_data_samce = pd.DataFrame([{
    'Attribute': [],
    'Absolute difference': [],
    'Sterilized mean': [],
    'Intact mean': [],
    'Bigger value is': []
}])

for i in info_rated['COLUMN NAME']:
    ster = data_ster_samce[i].mean()
    not_ster = data_notster_samce[i].mean()
    if ster > not_ster:
        bigger_value = 'sterilized'
    else:
        bigger_value = 'intact'
    ranked_data_samce.loc[len(ranked_data_samce)] = [i, abs(ster - not_ster), ster, not_ster, bigger_value]
    if abs(ster - not_ster) > max:
        max = abs(ster - not_ster)
        max_sterilized = ster
        max_intact = not_ster
        max_column = i
ranked_data_samce = ranked_data_samce.drop(index=0).sort_values(by='Absolute difference', ascending=False)

In [ ]:
all_samice = sns.barplot(data = ranked_data_samice, x='Attribute', y='Absolute difference', hue='Bigger value is')
all_samice.tick_params(axis='x', which='both', bottom=False, labelbottom=False)
plt.title("Biggest trait differences - Female")


In [ ]:
all_samce = sns.barplot(data = ranked_data_samce, x='Attribute', y='Absolute difference', hue='Bigger value is')
all_samce.tick_params(axis='x', which='both', bottom=False, labelbottom=False)
plt.title("Biggest trait differences - Male")

In [ ]:
intact_barplot_samice = sns.barplot(data = ranked_data_samice[ranked_data_samice['Bigger value is'] == 'intact'][:5], x='Attribute', y='Absolute difference')
sterilized_barplot_samice = sns.barplot(data = ranked_data_samice[ranked_data_samice['Bigger value is'] == 'sterilized'][:5], x='Attribute', y='Absolute difference')
plt.xticks(rotation=45, fontsize=7)
plt.title("Most differing attributes by sterilization status")
plt.xlabel("")

In [ ]:
intact_barplot_samce = sns.barplot(data = ranked_data_samce[ranked_data_samce['Bigger value is'] == 'intact'][:5], x='Attribute', y='Absolute difference')
sterilized_barplot_samce = sns.barplot(data = ranked_data_samce[ranked_data_samce['Bigger value is'] == 'sterilized'][:5], x='Attribute', y='Absolute difference')
plt.xticks(rotation=45, fontsize=7)
plt.title("Most differing attributes by sterilization status")
plt.xlabel("")

In [ ]:
rozdiely = pd.DataFrame([{
    'Attribut': [], 
    'rozdiel': [],
    'in_favor_of': []
}])
for i in ranked_data_samice['Attribute']:
    muz = ranked_data_samce[ranked_data_samce['Attribute'] == i]['Absolute difference'].values[0]
    fen = ranked_data_samice[ranked_data_samice['Attribute'] == i]['Absolute difference'].values[0]
    if muz > fen:
        naj = 'male'
    else:
        naj = 'female'
    rozdiely.loc[len(rozdiely)] = [i, abs(muz - fen), naj]

rozdiely = rozdiely.drop(index=0).sort_values(by='rozdiel', ascending=False)


In [ ]:

graf_rozdiel = sns.barplot(data = sorted_rozdiely[:15], x='Attribut', y='rozdiel', hue='in_favor_of')
plt.xticks(rotation=45, fontsize=7)